In [56]:
import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import seaborn as sns

In [57]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [59]:
data_dir = (Path().cwd().parents[0] / 'data'/'13cyc').absolute() / 'graphs'


In [60]:
import PPIGraph

In [61]:
condition_mapping = {'control': 0, '100nM': 1}

# Read per cell data

In [62]:
from torch_geometric.loader import DataLoader

In [63]:
# Filter out by maximum number of counts per cell
min_count = 20
max_count = 70

dataset = PPIGraph.GraphDataset(data_dir, 'raw', 'pt',  condition_mapping=condition_mapping, n_c=2)

# Create Dataloader
loader = DataLoader(dataset, batch_size=1, shuffle=False)

# Get Indices
indices = []
for step, data in enumerate(loader):
    if len(data.x) <= min_count:
        continue 
    
    if (data.x.sum(axis=0) >= max_count).any():
        continue
    indices.append(step)
    
# Get subset dataset
dataset_filtered = dataset.index_select(indices)


In [64]:
dataset

GraphDataset(2230)

In [65]:
dataset_filtered

GraphDataset(2117)

# ML model

In [66]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing, metrics
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, KFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler
import wandb

In [67]:
condition = 'ML'
project_name = f'PLA_13PPI_121923_{condition}'

In [68]:
models = {
    'Adaboost': AdaBoostClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'GradientBoosting' : GradientBoostingClassifier(),
    'NaiveBayes': GaussianNB(),
    'RandomForest': RandomForestClassifier(), 
    'SVM': SVC(probability =True),
    'LogisticRegression':  LogisticRegression(),
    'MLP': MLPClassifier(random_state=1, max_iter=100, hidden_layer_sizes=[16, 16, 16])
}



In [69]:
import torch 

data = torch.empty((0, 13))
label = []

for graph in dataset_filtered:
    new_row =graph.x.sum(axis = 0).unsqueeze(0)
    data = torch.cat((data, new_row), dim=0)
    label.append(graph.condition)


In [70]:
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import HistGradientBoostingClassifier


In [71]:
scaler = StandardScaler()

# Run model on cell count
X = data.numpy()
X = scaler.fit_transform(X)
y = np.array(label)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
kfold = KFold(n_splits = 5, shuffle = True, random_state = 0)

for model_name, model in models.items():
    for k, (train_index, test_index) in enumerate(kfold.split(X)):
        # Split the dataset
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probas = model.predict_proba(X_test)

        run = wandb.init(project=project_name, group=model_name+'_cell', name=model_name+f'_cell_{k}')

        accuracy = metrics.accuracy_score(y_test, y_pred)
        b_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)
        f1 = metrics.f1_score(y_test, y_pred)
        auc = metrics.roc_auc_score(y_test, y_pred)
        wandb.log({"accuracy": accuracy, 'b_accuracy': b_accuracy, 'f1':f1, 'auc': auc})
    run.finish()

accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.83726
auc,0.83714
b_accuracy,0.83714
f1,0.83373


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.75708
auc,0.75551
b_accuracy,0.75551
f1,0.73522


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.80851
auc,0.80667
b_accuracy,0.80667
f1,0.82276


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.85106
auc,0.8511
b_accuracy,0.8511
f1,0.85315


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.8227
auc,0.82275
b_accuracy,0.82275
f1,0.82185


...

c:\Users\thu71\AppData\Local\anaconda3\envs\PLA2\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.81797
auc,0.81681
b_accuracy,0.81681
f1,0.82851


c:\Users\thu71\AppData\Local\anaconda3\envs\PLA2\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.86525
auc,0.86528
b_accuracy,0.86528
f1,0.86651


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.8227
auc,0.82281
b_accuracy,0.82281
f1,0.8227
